<a href="https://colab.research.google.com/github/jkhall/grammar_error_detection/blob/master/CNN_GED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!ls /gdrive/My\ Drive/data


'aesw2016(v1.2)_train.xml'	 glove.6B.300d.txt
'aesw2016(v1.2)_train.xml.bz2'	 glove.6B.50d.txt
 glove.6B.100d.txt		 GoogleNews-vectors-negative300.bin
 glove.6B.200d.txt		 tarantino.csv


In [0]:
!unzip /gdrive/My\ Drive/data/glove\.6B\.zip -d /gdrive/My\ Drive/data/

Archive:  /gdrive/My Drive/data/glove.6B.zip
  inflating: /gdrive/My Drive/data/glove.6B.50d.txt  
  inflating: /gdrive/My Drive/data/glove.6B.100d.txt  
  inflating: /gdrive/My Drive/data/glove.6B.200d.txt  
  inflating: /gdrive/My Drive/data/glove.6B.300d.txt  


In [0]:
!gzip -dk /gdrive/My\ Drive/data/GoogleNews-vectors-negative300.bin.gz

In [3]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse('/gdrive/My Drive/data/aesw2016(v1.2)_train.xml')
root = tree.find('training')

# labels -> 0 = no error, 1 = error

par = root.find('par')
sentence = par.findall('sentence')

inputs = []
labels = []

def clean(src):
#   ins = src.iterfind('ins')
#   dels = src.findall('del')
  
  itr = src.iter()
  
  txt = ""
  
  if src.text != None:
    txt += src.text
 
  for tag in itr:
    if tag.tag == 'del':
      if tag.text != None:
        txt += tag.text
    
    if tag.tail != None:
      txt += tag.tail

  txt = re.sub('\n\t', " ", txt)
  txt = re.sub("_", "", txt)
  return txt
  

# err = sentence[1].find('del') == None #<-- this is how we check if the sentence contains errors

# create labels and inputs
for par in root.findall('par'):
  for sentence in par.findall('sentence'):
    inputs.append(clean(sentence))
    
    if sentence.find('del') != None or sentence.find('ins') != None:
      labels.append(1)
    else:
      labels.append(0)
      


print(inputs[2])
print(labels[2])
    
  

This is important since only MATH is the real, ordering parameter, while MATH is complex and thus has no natural ordering.  
0


In [0]:
s = sum(labels)
sentence_lengths = [len(sent.split()) for sent in inputs]
sl_total = sum(sentence_lengths)


print("Percentage of sentences with errors: {}".format(s / len(labels)))
print("Num examples: {}".format(len(inputs)))
print("Average sentence length: {}".format(sl_total / len(inputs)))

Percentage of sentences with errors: 0.3923552141730536
Num examples: 1189412
Average sentence length: 20.858449385074305


In [0]:
t = "hey,"
thing = "how do you do"
punc = "[!:;,.?]"
cnt = 7
arr = [1,2,3]
a = arr
cnt += 1
arr.append(4)
b = arr
# print("a: {}, b: {}".format(a, b))

print("in" in "print")

re.sub(punc, "", t) == t

True


False

In [4]:
# from tensorflow import keras
from keras.preprocessing import sequence, text
import numpy as np
import re

# Preprocessing
punc = '!?;:,.'

num_examples = 500000

# unclear how sensitive to vocab_size the model is
vocab_size = 10000

max_len = 40

# should get the most common words
# how should I deal with punctuation? Throw it all out? Try to keep it as a word? Embed it?
# can I have just a general <punc> tag? (eh)

def build_word_ids(sentences, vocabsize=1000):
  d = {}
  for s in sentences:
    words = s.split()
    for w in words:
      if w in d:
        d[w] += 1
      else:
        d[w] = 1
      
  sorted_d = sorted(d.items(), key=lambda x: x[1], reverse=True)
  
  w_ids = {}
  cnt = 1
  
  for k, v in sorted_d[:vocabsize]:
   
    w_ids[k] = cnt 
    cnt += 1

  return w_ids

word_ids = build_word_ids(inputs, vocabsize=vocab_size)
word_ids["<UKN>"] = vocab_size
vocab_size += 1
word_ids["!"] = vocab_size
vocab_size += 1
word_ids["?"] = vocab_size
vocab_size += 1
word_ids[";"] = vocab_size
vocab_size += 1
word_ids[":"] = vocab_size
vocab_size += 1
word_ids[","] = vocab_size
vocab_size += 2 # for the padding character

processed_inp = []

for sentence in inputs:
  words = sentence.split()
  new_sent = []
  for w in words:
    for p in punc:
      if p in w:
        new_sent.append(word_ids[p])
      w = re.sub(punc, "", w)
      break
    if w in word_ids:
      new_sent.append(word_ids[w])
    else:
      new_sent.append(word_ids["<UKN>"])
      
  processed_inp.append(new_sent)

# pad sequence with unused id: vocab_size - 2
processed_inp = sequence.pad_sequences(processed_inp, max_len)
  
X = np.array(processed_inp[:num_examples])
y = np.array(labels[:num_examples])

# shuffle
indices = np.arange(y.shape[0])
indices = np.random.shuffle(indices)

X = np.squeeze(X[indices])
y = np.squeeze(y[indices])

print(X.shape)
print(y.shape)

train_split = int(num_examples * 0.8)
# test_split = num_examples * 0.2

(X_train, y_train), (X_test, y_test) = (X[:train_split], y[:train_split]), (X[train_split:], y[train_split:])

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)


Using TensorFlow backend.


(500000, 40)
(500000,)
(400000, 40)
(100000, 40)
(400000,)


In [0]:
import numpy as np
import gensim
from gensim.models.keyedvectors import KeyedVectors

embedding_size = 300
embeddings1 = {}
embeddings2 = {}

glovePath = '/gdrive/My Drive/data/glove.6B.' + str(embedding_size) + 'd.txt'
word2VecPath = '/gdrive/My Drive/data/GoogleNews-vectors-negative300.bin'

embeddings2 = KeyedVectors.load_word2vec_format('/gdrive/My Drive/data/GoogleNews-vectors-negative300.bin', binary=True)
# word_vectors.save_word2vec_format('/gdrive/My Drive/data/GoogleNews-vectors-negative300.txt', binary=False)


f = open(glovePath)
for line in f:
  vals = line.split()
  word = vals[0]
  dimens = np.asarray(vals[1:], dtype='float32')
  embeddings1[word] = dimens
  
f.close()

embedding_mat1 = np.zeros((vocab_size, embedding_size))
embedding_mat2 = np.zeros((vocab_size, embedding_size))

for w, i in word_ids.items(): 
  # what else might we initialize absent words to, instead of zero?
  if w in embeddings1:
    embedding_mat1[i] = embeddings1[w]
    
for w, i in word_ids.items(): 
  # what else might we initialize absent words to, instead of zero?
  if w in embeddings2:
    embedding_mat2[i] = embeddings2[w]

In [0]:
print(X_train[0])

[    0     0     0     0     0   127  4724    26  3176  1178  1941     1
   216    14   251 15007     4  2319   149    10   300    51  1828    11
    32    30    19  1175     3    10     3     4   752  1993   167     3
     4     3    10    12]


In [8]:
import keras
from keras import layers
from keras.models import Sequential, Model

filter_sizes = [3,4,5]

inputs = layers.Input(shape=(max_len,))
emb1 = layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat1], trainable=True)(inputs)
emb2 = layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat2], trainable=False)(inputs)
c_emb = layers.concatenate([emb1, emb2])
# emb = layers.Embedding(vocab_size, embedding_size)(inputs)
conv1 = layers.Conv1D(128, filter_sizes[0], activation='relu')(c_emb)
pool1 = layers.MaxPool1D(max_len - filter_sizes[0] + 1)(conv1)
conv2 = layers.Conv1D(128, filter_sizes[1],activation='relu')(c_emb)
pool2 = layers.MaxPool1D(max_len - filter_sizes[1] + 1)(conv2)
conv3 = layers.Conv1D(128, filter_sizes[2],activation='relu')(c_emb)
pool3 = layers.MaxPool1D(max_len - filter_sizes[2] + 1)(conv3)
merged = layers.concatenate([pool1, pool2, pool3])
glo_pool = layers.GlobalMaxPooling1D()(merged)
drop = layers.Dropout(0.2)(glo_pool)
score = layers.Dense(1, activation='sigmoid')(drop)

model = Model(inputs=inputs, outputs=score)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

model.summary()

history = model.fit(X_train, y_train,
         epochs=2,
         batch_size=100,
         validation_split=0.2)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 40, 300)      3002100     input_2[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 40, 300)      3002100     input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 40, 600)      0           embedding_3[0][0]                
                                                                 embedding_4[0][0]                
__________

In [0]:
import keras 
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat], trainable=False))
model.add(layers.LSTM(30, return_sequences=True))
model.add(layers.LSTM(30, return_sequences=True, go_backwards=True))
model.add(layers.LSTM(30))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

model.summary()

history = model.fit(X_train, y_train,
                   epochs=10,
                   batch_size=100,
                   validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 30)          15720     
_________________________________________________________________
lstm_14 (LSTM)               (None, None, 30)          7320      
_________________________________________________________________
lstm_15 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 2,030,391
Trainable params: 30,391
Non-trainable params: 2,000,000
_____________________________________________________________

In [0]:
import keras 
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat], trainable=False))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(32))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

model.summary()

history = model.fit(X_train, y_train,
                   epochs=20,
                   batch_size=100,
                   validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 32)          17024     
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,033,697
Trainable params: 33,697
Non-trainable params: 2,000,000
_____________________________________________________________

In [12]:
import keras
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat1], trainable=False))
model.add(layers.LSTM(128))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

model.summary()

history = model.fit(X_train, y_train,
                   epochs=10,
                   batch_size=100,
                   validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 300)         3002100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 3,221,877
Trainable params: 219,777
Non-trainable params: 3,002,100
_________________________________________________________________
Train on 320000 samples, validate on 80000 samples
Epoch 1/10
320000/320000 [==============================] - 401s 1ms/step - loss: 0.5929 - acc: 0.6770 - val_loss: 0.5748 - val_acc: 0.6949
Epoch 2/10
320000/320000 [==============================] - 396s 1m

In [11]:
import keras
from keras import layers
from keras.models import Sequential
from keras.optimizers import RMSprop


model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_size, weights=[embedding_mat1], trainable=False))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
             loss='binary_crossentropy',
             metrics=['acc'])

history = model.fit(X_train, y_train,
                   epochs=10,
                   batch_size=100,
                   validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 300)         3002100   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 64)          57664     
_________________________________________________________________
conv1d_12 (Conv1D)           (None, None, 64)          12352     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, None, 64)          12352     
_________________________________________________________________
conv1d_14 (Conv1D)           (None, None, 64)          12352     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
__________